In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Load the dataset
df = pd.read_csv('../data/hello_world_dataset.csv')

# Convert greetings to numerical labels
greetings = df['Greeting'].unique()
greeting_to_label = {greeting: label for label, greeting in enumerate(greetings)}
df['Label'] = df['Greeting'].map(greeting_to_label)

# Prepare features and labels
X = df['Greeting'].values.reshape(-1, 1)  # Features (reshape for OneHotEncoder)
y = df['Label'].values                    # Labels

# One-hot encode the features
encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(X)

# Define a simple neural network
class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.weights1 = np.random.randn(input_size, hidden_size) * 0.01
        self.bias1 = np.zeros((1, hidden_size))
        self.weights2 = np.random.randn(hidden_size, output_size) * 0.01
        self.bias2 = np.zeros((1, output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, X):
        self.hidden_input = np.dot(X, self.weights1) + self.bias1
        self.hidden_output = self.sigmoid(self.hidden_input)
        self.output_input = np.dot(self.hidden_output, self.weights2) + self.bias2
        self.output = self.softmax(self.output_input)
        return self.output

    def train(self, X, y, epochs=1000, learning_rate=0.1):
        for epoch in range(epochs):
            output = self.forward(X)
            m = y.shape[0]
            log_probs = -np.log(output[range(m), y])
            loss = np.sum(log_probs) / m

            d_output = output
            d_output[range(m), y] -= 1
            d_output /= m
            d_hidden = np.dot(d_output, self.weights2.T) * (self.hidden_output * (1 - self.hidden_output))

            self.weights2 -= learning_rate * np.dot(self.hidden_output.T, d_output)
            self.bias2 -= learning_rate * np.sum(d_output, axis=0, keepdims=True)
            self.weights1 -= learning_rate * np.dot(X.T, d_hidden)
            self.bias1 -= learning_rate * np.sum(d_hidden, axis=0, keepdims=True)

            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)

# Create and train the neural network
input_size = X_encoded.shape[1]
output_size = len(greetings)
nn = SimpleNeuralNetwork(input_size, hidden_size=10, output_size=output_size)
nn.train(X_encoded, y, epochs=1000, learning_rate=0.1)

# Example prediction
new_greeting = "Hello, World!"
new_greeting_encoded = encoder.transform(np.array([new_greeting]).reshape(-1, 1))
predicted_label = nn.predict(new_greeting_encoded)
predicted_greeting = greetings[predicted_label[0]]
print(f"Predicted greeting: {predicted_greeting}")


Epoch 0, Loss: 2.1973083534972235
Epoch 100, Loss: 2.197088155728292
Epoch 200, Loss: 2.196929584832424
Epoch 300, Loss: 2.196679113586409
Epoch 400, Loss: 2.1962586566654774
Epoch 500, Loss: 2.195536983167641
Epoch 600, Loss: 2.1942889062928206
Epoch 700, Loss: 2.192126000123629
Epoch 800, Loss: 2.188379190860583
Epoch 900, Loss: 2.1819028262069757
Predicted greeting: Hello, World!
